In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Model
#print(tf.keras.__version__)
#print(tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

In [ ]:
dir = "D:/WindData/POD_Request_3541/Project/Copies/processed/trimmed/final"

os.chdir(dir)


In [2]:
visible = layers.Input(shape=(60, 573,))

cnn1 = layers.Conv1D(150, 3, activation='relu')(visible)
cnn2 = layers.Conv1D(300, 3, activation='relu')(cnn1)
pool = layers.MaxPooling1D(6)(cnn2)
drop = layers.Dropout(0.25)(pool)
flat = layers.Flatten()(pool)

lstm1 = layers.LSTM(150, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(visible)
lstm2 = layers.LSTM(300, dropout=0.1, recurrent_dropout=0.1)(lstm1)

combo = layers.merge.concatenate([flat,lstm2])
dense = layers.Dense(100, activation='relu')(combo)
out = layers.Dense(1)(dense)

model = Model(inputs=visible, outputs=out)




print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 573)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 58, 150)      258000      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 56, 300)      135300      conv1d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 9, 300)       0           conv1d_2[0][0]                   
____________________________________________________________________________________________

In [3]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='mse', 
              metrics=['mae', 'mse'])

In [ ]:
batchCount = 150

availableBatch = range(batchCount)

labelFrame = pd.read_csv("labels.csv")
labelList = labelFrame.values.tolist()


def generator():
    if len(availableBatch)==0:
        availableBatch = range(batchCount)
    nextIndex = rng.randint(0, len(availableBatch))
    nextIndex = availableBatch.pop(nextIndex)
    targets = labelList[nextIndex]
    data = np.load("data/" + str(selected) + ".npy")
    return (data, targets)

In [ ]:
model.fit_generator(generator(), steps_per_epoch=150, epochs=5)